In [1]:
import argparse
from argparse import Namespace
import os
# workaround to unpickle olf model files
import sys
import time
from tqdm import tqdm_notebook as tqdm

import numpy as np
import torch

from a2c_ppo_acktr.envs import VecPyTorch, make_vec_envs
from a2c_ppo_acktr.utils import get_render_func, get_vec_normalize

%matplotlib tk
import matplotlib.pyplot as plt

from mujoco_py import GlfwContext
GlfwContext(offscreen=True)

sys.path.append('a2c_ppo_acktr')

/is/ei/fleeb/anaconda3/envs/testmj/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/is/ei/fleeb/anaconda3/envs/testmj/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/is/ei/fleeb/anaconda3/envs/testmj/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/is/ei/fleeb/anaconda3/en

Creating offscreen glfw


In [2]:
args = Namespace()
args.seed = 0
args.log_interval = 10
args.det = False
args.root = '/is/ei/fleeb/workspace/clones/pytorch-a2c-ppo-acktr-gail'
args.load_root = os.path.join(args.root, 'trained_models')

args.batch_size = 4

args.no_state = True
args.max_steps = 1000

args.alg = 'ppo'
args.alg = 'acktr'

args.load_dir = os.path.join(args.load_root, args.alg)

args.env_name = 'HalfCheetah-v2'
# args.env_name = 'BreakoutNoFrameskip-v4'
args.env_name = 'SeaquestNoFrameskip-v4'
args.env_name = 'AirRaidNoFrameskip-v4'
args.env_name = 'AsterixNoFrameskip-v4'
args.env_name = 'SpaceInvadersNoFrameskip-v4'
args.env_name = 'AsteroidsNoFrameskip-v4'
args.env_name = 'QbertNoFrameskip-v4'
args.env_name = 'MsPacmanNoFrameskip-v4'

In [3]:
# import gym
# from gym.utils.play import play

# env_name = 'QbertNoFrameskip-v4'

# play(gym.make(env_name), zoom=4)


In [4]:
def play_back(imgs, figax=None):
    
    if len(imgs.shape) > 4: # tile first
        N, S, h, w, c = imgs.shape
        H = int(np.ceil(np.sqrt(N)))
        W = int(np.ceil(float(N)/H))
        imgs = imgs.reshape(H,W,S,h,w,c)
        imgs = imgs.transpose(2,0,3,1,4,5)
        imgs = imgs.reshape(S,H*h,W*w,c)
        
    if figax is None:
        figax = plt.subplots()
    fig,ax = figax
    plt.sca(ax)
    for i,img in enumerate(imgs):
        plt.cla()
        plt.axis('off')
        plt.imshow(img)
        plt.title('{}/{}'.format(i+1,len(imgs)))
        plt.tight_layout()
        plt.pause(0.02)
    return fig,ax

In [5]:
env = make_vec_envs(
    args.env_name,
    args.seed + 1000,
    args.batch_size,
    None,
    None,
    device='cpu',
    allow_early_resets=False)

# Get a render function
render_func = get_render_func(env)

render_func = env.render

# We need to use the same statistics for normalization as used in training
actor_critic, ob_rms = \
            torch.load(os.path.join(args.load_dir, args.env_name + ".pt"))

vec_norm = get_vec_normalize(env)
if vec_norm is not None:
    vec_norm.eval()
    vec_norm.ob_rms = ob_rms

recurrent_hidden_states = torch.zeros(args.batch_size,
                                      actor_critic.recurrent_hidden_state_size)
masks = torch.zeros(args.batch_size, 1)




Logging to /tmp/openai-2020-01-16-12-15-22-291538
Creating dummy env object to get spaces


In [10]:
playfigax = None

In [7]:
obs = env.reset()

if args.env_name.find('Bullet') > -1:
    import pybullet as p

    torsoId = -1
    for i in range(p.getNumBodies()):
        if (p.getBodyInfo(i)[0].decode() == "torso"):
            torsoId = i

if render_func is not None:
    rgb = render_func('rgb_array')
    rgbs = [rgb]
else:
    rgbs = None
    print('No rendering possible')
            
states = None if args.no_state else [obs]
actions = []
rewards = []

pbar = tqdm(total=args.max_steps)

done = np.array([False])
step = 0
while not done.all() and (args.max_steps is None or step < args.max_steps):
    with torch.no_grad():
        value, action, _, recurrent_hidden_states = actor_critic.act(
            obs, recurrent_hidden_states, masks, deterministic=args.det)

    actions.append(action)
        
    # Obser reward and next obs
    obs, reward, done, _ = env.step(action)
    step += 1
    pbar.update(1)
    if states is not None:
        states.append(obs)
    rewards.append(reward)
        
    masks.copy_(torch.from_numpy(~done).unsqueeze(1))

    if args.env_name.find('Bullet') > -1:
        if torsoId > -1:
            distance = 5
            yaw = 0
            humanPos, humanOrn = p.getBasePositionAndOrientation(torsoId)
            p.resetDebugVisualizerCamera(distance, yaw, -20, humanPos)

    if render_func is not None:
        rgb = render_func('rgb_array')
        rgbs.append(rgb)


# actions = torch.stack(actions,1)
# print('actions', actions.shape)
# rewards = torch.stack(rewards,1)
# print('rewards', rewards.shape)
# if states is not None:
#     states = torch.cat(states)
#     print('states', states.shape)
if rgbs is not None:
    rgbs = np.stack(rgbs,1)
    print('rgbs', rgbs.shape)

/is/ei/fleeb/anaconda3/envs/testmj/lib/python3.6/site-packages/ipykernel_launcher.py:22: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


rgbs (4, 1001, 210, 160, 3)


In [11]:
playfigax = play_back(rgbs, figax=playfigax)

In [13]:
!du -hs test-data/*

376M	test-data/Asterix
358M	test-data/Seaquest
334M	test-data/SpaceInvaders


In [ ]:
done

In [12]:
masks

tensor([[1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.]])

In [19]:
import h5py as hf

In [25]:
root = '/is/ei/fleeb/workspace/clones/pytorch-a2c-ppo-acktr-gail/'

In [35]:
f = hf.File(os.path.join(root, 'test-data', 'AsterixNoFrameskip-v4_16x1024_batch0.h5'),'r')
list(f.keys())

['images']

In [36]:
f['images'].shape

(16, 1024)

In [37]:
raw = f['images'][()]
raw.shape

(16, 1024)

In [40]:
img = util.str_to_rgb(raw[0,50])

In [41]:
img = img[::2,::2]

In [43]:
img.shape

(105, 80, 3)

In [42]:
plt.imshow(img)